In [20]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [21]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [22]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [23]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [24]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data)

In [25]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(df_market_data_scaled, columns=df_market_data.columns, index=df_market_data.index)
# Display sample data
df_market_data_scaled.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [26]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [27]:
# Create an empty list to store the inertia values
inertia = []
# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=0)
    # 2. Fit the model to the data using `df_market_data_scaled`   
    model.fit(df_market_data_scaled)
    # 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

In [28]:
# Create a dictionary with the data to plot the Elbow curve
data = {"k": k_values, "inertia": inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_inertia = pd.DataFrame(data)

In [29]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_inertia.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_values,
    width=800,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [30]:
# Initialize the K-Means model using the best value for k
k = 4
model = KMeans(n_clusters=k, random_state=0)

In [31]:
# Fit the K-Means model using the scaled data
kmeans = model.fit(df_market_data_scaled)

In [32]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_values = kmeans.predict(df_market_data_scaled)
# Print the resulting array of cluster values.
print(cluster_values)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [33]:
# Create a copy of the DataFrame
df_market_data_clustered = df_market_data.copy()

In [36]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_clustered["predicted_clusters"] = kmeans.labels_
# Display sample data
df_market_data_clustered.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_clusters
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,2
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,2
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,2


In [38]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

df_market_data_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols=["coin_name"],
    title="Crypto Market Clusters",
    color="predicted_clusters"
)


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_clusters)

---

### Optimize Clusters with Principal Component Analysis.

In [39]:
# Create a PCA model instance and set `n_components=3`.
pca =  PCA(n_components=3)

In [42]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)
market_data_pca = pd.DataFrame(
    data=market_data_pca, columns=["principal component 1", "principal component 2", "principal component 3"], index=df_market_data.index
)
# View the first five rows of the DataFrame.
market_data_pca.head(5)

,principal component 1,principal component 2,principal component 3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [47]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
explained_variance
#total_explained_variance = explained_variance[0] + explained_variance[1] + explained_variance[2]
#total_explained_variance

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.50%

In [49]:
# Create a new DataFrame with the PCA data.
# Copy the crypto names from the original data
# Set the coinid column as index
# Display sample data
market_data_pca.head(5)

,principal component 1,principal component 2,principal component 3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [50]:
# Create a list with the number of k-values from 1 to 11
pca_k_values = list(range(1, 12))

In [51]:
# Create an empty list to store the inertia values
pca_inertia = []
# Create a for loop to compute the inertia with each possible value of k
for k in pca_k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    pca_model = KMeans(n_clusters=k, random_state=0)
# 2. Fit the model to the data using `df_market_data_pca`
    pca_model.fit(market_data_pca)
# 3. Append the model.inertia_ to the inertia list
    pca_inertia.append(pca_model.inertia_)

In [52]:
# Create a dictionary with the data to plot the Elbow curve
pca_data = {"k": pca_k_values, "inertia": pca_inertia}
# Create a DataFrame with the data to plot the Elbow curve
pca_df_inertia = pd.DataFrame(pca_data)

In [53]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_df_inertia.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve with PCA",
    xticks=pca_k_values,
    width=800,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [54]:
# Initialize the K-Means model using the best value for k
pca_k = 4
pca_model = KMeans(n_clusters=pca_k, random_state=0)

In [55]:
# Fit the K-Means model using the PCA data
pca_kmeans = pca_model.fit(market_data_pca)

In [57]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_cluster_values = pca_kmeans.predict(market_data_pca)
# Print the resulting array of cluster values.
print(pca_cluster_values)


[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [58]:
# Create a copy of the DataFrame with the PCA data
df_market_data_pca_clustered = market_data_pca.copy()
# Add a new column to the DataFrame with the predicted clusters
df_market_data_pca_clustered["predicted_clusters"] = pca_kmeans.labels_
# Display sample data
df_market_data_pca_clustered.head(5)

,principal component 1,principal component 2,principal component 3,predicted_clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [59]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_pca_clustered.hvplot.scatter(
    x="principal component 1",
    y="principal component 2",
    hover_cols=["coin_name"],
    title="Crypto Market Clusters with PCA",
    color="predicted_clusters"
)

:Scatter   [principal component 1]   (principal component 2,predicted_clusters)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [61]:
# Composite plot to contrast the Elbow curves
plot1 = df_inertia.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_values,
    width=400,
    height=400
)
plot2 = pca_df_inertia.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve with PCA",
    xticks=pca_k_values,
    width=400,
    height=400
)
composite_plot = plot1 + plot2
composite_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [62]:
# Composite plot to contrast the clusters
plot1 = df_market_data_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols=["coin_name"],
    title="Crypto Market Clusters",
    color="predicted_clusters"
)
plot2 = df_market_data_pca_clustered.hvplot.scatter(
    x="principal component 1",
    y="principal component 2",
    hover_cols=["coin_name"],
    title="Crypto Market Clusters with PCA",
    color="predicted_clusters"
)
composite_plot = plot1 + plot2
composite_plot

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_clusters)
   .Scatter.II :Scatter   [principal component 1]   (principal component 2,predicted_clusters)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 